<a href="https://colab.research.google.com/github/dinuka-rp/nerdy-panda/blob/main/Prototype/RecSys/Trends_Content_based_NFT_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Content-based NFT Recommender System

https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

Check this later (not that different): https://medium.com/dataseries/recommendation-system-in-python-4d4cf6d33166 , https://gist.github.com/deansublett/06f010a886831f9dac5b1f9bce4f4229?source=post_page---------------------------

In [ ]:
!pip install rake-nltk

     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 748 kB 31.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Get Dataset

### NFT Data

In [75]:
from google.colab import drive
drive.mount('drive')

assets_file_path = "/content/drive/MyDrive/Datasets/FYP/collected-nft-assets"
df = pd.read_csv(assets_file_path, sep='\t')

# df = df[['nft_id','traits_string','asset_contract_address', 'total_rarity']]
df.head()

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


,nft_id,name,asset_description,asset_contract_address,collection_name,collection_description,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url
0,2541585283815458503018281017495372409420154545...,CelebCard #84 Snoop Dogg,NaN,0x495f947276749ce646f68ac8c248420045cb7b5e,Mesmerizer,No theme. No traits. No rules. A treasure trov...,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...
1,4460258260744616717349398633449740654585409544...,3D Penguin Planet Official Collection,3D Penguin Planet Collection It was hand paint...,0x495f947276749ce646f68ac8c248420045cb7b5e,3D Penguin Planet Official Collection,3D Penguin Planet Official Collection It was h...,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...
2,1143238369460653929904223751867794581426030911...,VECTOR KAS 10 (The Relax),VECTOR KAS 10,0x495f947276749ce646f68ac8c248420045cb7b5e,Kalvintaka Collection,NaN,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...
3,5742295951199733757787373026863398866922647154...,Gooniez #1989,"A gang of 8,888 gooniez unleashed to wreak hav...",0x495f947276749ce646f68ac8c248420045cb7b5e,Metaverse GooniezNFT,"A gang of 8,888 gooniez unleashed to wreak hav...",metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...
4,1006254684714400059060824656421626037946389019...,Darumaa Soul Story.#3,Works that are created by an Extraterrestrial ...,0x495f947276749ce646f68ac8c248420045cb7b5e,Extraterrestrial Soul Darumaa,Darumaa. Not Real & Reality.\n\nWorks that are...,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...


### Trends Data

In [76]:
auth_token = "AAAAAAAAAAAAAAAAAAAAAEAvTAEAAAAAQeWT9FEvcpxz3Ebg5IS45oASCjM%3DT67USoXi0iCtPoLRMHpDGQw3unxzJlEXHY2CZWR04UcCvtTcmn"

In [77]:
import requests

def fetch_twitter_trends(woeid=1):

  url = "https://api.twitter.com/1.1/trends/place.json?id=" + str(woeid)

  payload={}
  headers = {
    'Authorization': 'Bearer ' + auth_token,
    'Cookie': 'guest_id=v1%3A164302265238123185; guest_id_ads=v1%3A164302265238123185; guest_id_marketing=v1%3A164302265238123185; personalization_id="v1_7SOW+P3aER3WzxCMuCXiWw=="'
  }

  response = requests.request("GET", url, headers=headers, data=payload)

  if response.status_code != 200:
    print('error')

  trends = response.json()[0]['trends']
  # print(trends)
  return trends

twitter_trends = fetch_twitter_trends()
# twitter_trends = fetch_twitter_trends(woeid=2282863)

print(twitter_trends)

[{'name': 'FIFA', 'url': 'http://twitter.com/search?q=FIFA', 'promoted_content': None, 'query': 'FIFA', 'tweet_volume': 322831}, {'name': '#TheBatman', 'url': 'http://twitter.com/search?q=%23TheBatman', 'promoted_content': None, 'query': '%23TheBatman', 'tweet_volume': 108136}, {'name': '#KASvFB', 'url': 'http://twitter.com/search?q=%23KASvFB', 'promoted_content': None, 'query': '%23KASvFB', 'tweet_volume': None}, {'name': 'Reinaldo', 'url': 'http://twitter.com/search?q=Reinaldo', 'promoted_content': None, 'query': 'Reinaldo', 'tweet_volume': None}, {'name': '#ARMYSelcaDay', 'url': 'http://twitter.com/search?q=%23ARMYSelcaDay', 'promoted_content': None, 'query': '%23ARMYSelcaDay', 'tweet_volume': 20144}, {'name': 'Qatar', 'url': 'http://twitter.com/search?q=Qatar', 'promoted_content': None, 'query': 'Qatar', 'tweet_volume': 61377}, {'name': '#ARSD', 'url': 'http://twitter.com/search?q=%23ARSD', 'promoted_content': None, 'query': '%23ARSD', 'tweet_volume': 21343}, {'name': 'Igor Gomes',

In [47]:
twitter_trends = [{'name': '#โหนกระเเส', 'url': 'http://twitter.com/search?q=%23%E0%B9%82%E0%B8%AB%E0%B8%99%E0%B8%81%E0%B8%A3%E0%B8%B0%E0%B9%80%E0%B9%80%E0%B8%AA', 'promoted_content': None, 'query': '%23%E0%B9%82%E0%B8%AB%E0%B8%99%E0%B8%81%E0%B8%A3%E0%B8%B0%E0%B9%80%E0%B9%80%E0%B8%AA', 'tweet_volume': 2166180}, {'name': '#Euphoria', 'url': 'http://twitter.com/search?q=%23Euphoria', 'promoted_content': None, 'query': '%23Euphoria', 'tweet_volume': 1005172}, {'name': 'Cassie', 'url': 'http://twitter.com/search?q=Cassie', 'promoted_content': None, 'query': 'Cassie', 'tweet_volume': 416023}, {'name': 'Maddy', 'url': 'http://twitter.com/search?q=Maddy', 'promoted_content': None, 'query': 'Maddy', 'tweet_volume': 352427}, {'name': '#28Şubat', 'url': 'http://twitter.com/search?q=%2328%C5%9Eubat', 'promoted_content': None, 'query': '%2328%C5%9Eubat', 'tweet_volume': 42633}, {'name': 'Elliot', 'url': 'http://twitter.com/search?q=Elliot', 'promoted_content': None, 'query': 'Elliot', 'tweet_volume': 231872}, {'name': 'Andalucía', 'url': 'http://twitter.com/search?q=Andaluc%C3%ADa', 'promoted_content': None, 'query': 'Andaluc%C3%ADa', 'tweet_volume': 39283}, {'name': '#MondayMotivation', 'url': 'http://twitter.com/search?q=%23MondayMotivation', 'promoted_content': None, 'query': '%23MondayMotivation', 'tweet_volume': 23904}, {'name': 'Lexi', 'url': 'http://twitter.com/search?q=Lexi', 'promoted_content': None, 'query': 'Lexi', 'tweet_volume': 246787}, {'name': '#Callin_MV_Out', 'url': 'http://twitter.com/search?q=%23Callin_MV_Out', 'promoted_content': None, 'query': '%23Callin_MV_Out', 'tweet_volume': 23107}, {'name': 'Lakers', 'url': 'http://twitter.com/search?q=Lakers', 'promoted_content': None, 'query': 'Lakers', 'tweet_volume': 89051}, {'name': 'Larissa', 'url': 'http://twitter.com/search?q=Larissa', 'promoted_content': None, 'query': 'Larissa', 'tweet_volume': 329769}, {'name': 'Jules', 'url': 'http://twitter.com/search?q=Jules', 'promoted_content': None, 'query': 'Jules', 'tweet_volume': 100974}, {'name': 'sam levinson', 'url': 'http://twitter.com/search?q=%22sam+levinson%22', 'promoted_content': None, 'query': '%22sam+levinson%22', 'tweet_volume': 93293}, {'name': 'Faye', 'url': 'http://twitter.com/search?q=Faye', 'promoted_content': None, 'query': 'Faye', 'tweet_volume': 107648}, {'name': 'Nate', 'url': 'http://twitter.com/search?q=Nate', 'promoted_content': None, 'query': 'Nate', 'tweet_volume': 137511}, {'name': 'Toz Taneleri', 'url': 'http://twitter.com/search?q=%22Toz+Taneleri%22', 'promoted_content': None, 'query': '%22Toz+Taneleri%22', 'tweet_volume': None}, {'name': "NU'EST", 'url': 'http://twitter.com/search?q=NU%27EST', 'promoted_content': None, 'query': 'NU%27EST', 'tweet_volume': 13042}, {'name': '政策金利', 'url': 'http://twitter.com/search?q=%E6%94%BF%E7%AD%96%E9%87%91%E5%88%A9', 'promoted_content': None, 'query': '%E6%94%BF%E7%AD%96%E9%87%91%E5%88%A9', 'tweet_volume': None}, {'name': '非核三原則', 'url': 'http://twitter.com/search?q=%E9%9D%9E%E6%A0%B8%E4%B8%89%E5%8E%9F%E5%89%87', 'promoted_content': None, 'query': '%E9%9D%9E%E6%A0%B8%E4%B8%89%E5%8E%9F%E5%89%87', 'tweet_volume': None}, {'name': 'やまとん先生', 'url': 'http://twitter.com/search?q=%E3%82%84%E3%81%BE%E3%81%A8%E3%82%93%E5%85%88%E7%94%9F', 'promoted_content': None, 'query': '%E3%82%84%E3%81%BE%E3%81%A8%E3%82%93%E5%85%88%E7%94%9F', 'tweet_volume': None}, {'name': 'Zendaya', 'url': 'http://twitter.com/search?q=Zendaya', 'promoted_content': None, 'query': 'Zendaya', 'tweet_volume': 158640}, {'name': 'Maddie', 'url': 'http://twitter.com/search?q=Maddie', 'promoted_content': None, 'query': 'Maddie', 'tweet_volume': 38518}, {'name': 'Ashtray', 'url': 'http://twitter.com/search?q=Ashtray', 'promoted_content': None, 'query': 'Ashtray', 'tweet_volume': None}, {'name': 'Tansu Çiller', 'url': 'http://twitter.com/search?q=%22Tansu+%C3%87iller%22', 'promoted_content': None, 'query': '%22Tansu+%C3%87iller%22', 'tweet_volume': None}, {'name': '立憲・末松義規議員', 'url': 'http://twitter.com/search?q=%E7%AB%8B%E6%86%B2%E3%83%BB%E6%9C%AB%E6%9D%BE%E7%BE%A9%E8%A6%8F%E8%AD%B0%E5%93%A1', 'promoted_content': None, 'query': '%E7%AB%8B%E6%86%B2%E3%83%BB%E6%9C%AB%E6%9D%BE%E7%BE%A9%E8%A6%8F%E8%AD%B0%E5%93%A1', 'tweet_volume': None}, {'name': 'ランファ', 'url': 'http://twitter.com/search?q=%E3%83%A9%E3%83%B3%E3%83%95%E3%82%A1', 'promoted_content': None, 'query': '%E3%83%A9%E3%83%B3%E3%83%95%E3%82%A1', 'tweet_volume': 31601}, {'name': '뉴이스트', 'url': 'http://twitter.com/search?q=%EB%89%B4%EC%9D%B4%EC%8A%A4%ED%8A%B8', 'promoted_content': None, 'query': '%EB%89%B4%EC%9D%B4%EC%8A%A4%ED%8A%B8', 'tweet_volume': 17313}, {'name': 'Tiziano Ferro', 'url': 'http://twitter.com/search?q=%22Tiziano+Ferro%22', 'promoted_content': None, 'query': '%22Tiziano+Ferro%22', 'tweet_volume': None}, {'name': 'ファイナルミキサー', 'url': 'http://twitter.com/search?q=%E3%83%95%E3%82%A1%E3%82%A4%E3%83%8A%E3%83%AB%E3%83%9F%E3%82%AD%E3%82%B5%E3%83%BC', 'promoted_content': None, 'query': '%E3%83%95%E3%82%A1%E3%82%A4%E3%83%8A%E3%83%AB%E3%83%9F%E3%82%AD%E3%82%B5%E3%83%BC', 'tweet_volume': None}, {'name': 'jisoo', 'url': 'http://twitter.com/search?q=jisoo', 'promoted_content': None, 'query': 'jisoo', 'tweet_volume': 340519}, {'name': 'Lina', 'url': 'http://twitter.com/search?q=Lina', 'promoted_content': None, 'query': 'Lina', 'tweet_volume': 107987}, {'name': 'Pledis', 'url': 'http://twitter.com/search?q=Pledis', 'promoted_content': None, 'query': 'Pledis', 'tweet_volume': 108854}, {'name': 'dominic fike', 'url': 'http://twitter.com/search?q=%22dominic+fike%22', 'promoted_content': None, 'query': '%22dominic+fike%22', 'tweet_volume': 15835}, {'name': 'ロシア大幅利上げ', 'url': 'http://twitter.com/search?q=%E3%83%AD%E3%82%B7%E3%82%A2%E5%A4%A7%E5%B9%85%E5%88%A9%E4%B8%8A%E3%81%92', 'promoted_content': None, 'query': '%E3%83%AD%E3%82%B7%E3%82%A2%E5%A4%A7%E5%B9%85%E5%88%A9%E4%B8%8A%E3%81%92', 'tweet_volume': None}, {'name': '異例の声明', 'url': 'http://twitter.com/search?q=%E7%95%B0%E4%BE%8B%E3%81%AE%E5%A3%B0%E6%98%8E', 'promoted_content': None, 'query': '%E7%95%B0%E4%BE%8B%E3%81%AE%E5%A3%B0%E6%98%8E', 'tweet_volume': None}, {'name': '日本の追加制裁', 'url': 'http://twitter.com/search?q=%E6%97%A5%E6%9C%AC%E3%81%AE%E8%BF%BD%E5%8A%A0%E5%88%B6%E8%A3%81', 'promoted_content': None, 'query': '%E6%97%A5%E6%9C%AC%E3%81%AE%E8%BF%BD%E5%8A%A0%E5%88%B6%E8%A3%81', 'tweet_volume': None}, {'name': 'minhyun', 'url': 'http://twitter.com/search?q=minhyun', 'promoted_content': None, 'query': 'minhyun', 'tweet_volume': None}, {'name': '通貨防衛', 'url': 'http://twitter.com/search?q=%E9%80%9A%E8%B2%A8%E9%98%B2%E8%A1%9B', 'promoted_content': None, 'query': '%E9%80%9A%E8%B2%A8%E9%98%B2%E8%A1%9B', 'tweet_volume': None}, {'name': '米ホワイトハウス', 'url': 'http://twitter.com/search?q=%E7%B1%B3%E3%83%9B%E3%83%AF%E3%82%A4%E3%83%88%E3%83%8F%E3%82%A6%E3%82%B9', 'promoted_content': None, 'query': '%E7%B1%B3%E3%83%9B%E3%83%AF%E3%82%A4%E3%83%88%E3%83%8F%E3%82%A6%E3%82%B9', 'tweet_volume': None}, {'name': 'ルーブル', 'url': 'http://twitter.com/search?q=%E3%83%AB%E3%83%BC%E3%83%96%E3%83%AB', 'promoted_content': None, 'query': '%E3%83%AB%E3%83%BC%E3%83%96%E3%83%AB', 'tweet_volume': 29897}, {'name': 'Laurie', 'url': 'http://twitter.com/search?q=Laurie', 'promoted_content': None, 'query': 'Laurie', 'tweet_volume': 45052}, {'name': 'Sadi Somuncuoğlu', 'url': 'http://twitter.com/search?q=%22Sadi+Somuncuo%C4%9Flu%22', 'promoted_content': None, 'query': '%22Sadi+Somuncuo%C4%9Flu%22', 'tweet_volume': None}, {'name': 'Wochenstart', 'url': 'http://twitter.com/search?q=Wochenstart', 'promoted_content': None, 'query': 'Wochenstart', 'tweet_volume': None}, {'name': 'HANGGANG SA MULI ALEIAN', 'url': 'http://twitter.com/search?q=%22HANGGANG+SA+MULI+ALEIAN%22', 'promoted_content': None, 'query': '%22HANGGANG+SA+MULI+ALEIAN%22', 'tweet_volume': 18804}, {'name': '中田翔選手', 'url': 'http://twitter.com/search?q=%E4%B8%AD%E7%94%B0%E7%BF%94%E9%81%B8%E6%89%8B', 'promoted_content': None, 'query': '%E4%B8%AD%E7%94%B0%E7%BF%94%E9%81%B8%E6%89%8B', 'tweet_volume': None}, {'name': 'ロシア中銀', 'url': 'http://twitter.com/search?q=%E3%83%AD%E3%82%B7%E3%82%A2%E4%B8%AD%E9%8A%80', 'promoted_content': None, 'query': '%E3%83%AD%E3%82%B7%E3%82%A2%E4%B8%AD%E9%8A%80', 'tweet_volume': None}, {'name': 'BENJI KUMUSERYE EP3', 'url': 'http://twitter.com/search?q=%22BENJI+KUMUSERYE+EP3%22', 'promoted_content': None, 'query': '%22BENJI+KUMUSERYE+EP3%22', 'tweet_volume': None}, {'name': 'Viny', 'url': 'http://twitter.com/search?q=Viny', 'promoted_content': None, 'query': 'Viny', 'tweet_volume': 25361}, {'name': 'Baekho', 'url': 'http://twitter.com/search?q=Baekho', 'promoted_content': None, 'query': 'Baekho', 'tweet_volume': 12652}]

## Data Cleaning

In [78]:
for index, row in df.iterrows():
    df.at[index,'reference_id'] = row["asset_contract_address"] + "-" + str(row["nft_id"])

# df['traits_string'] = df['traits_string'].str.replace(';',' ') # this needs to be changed in BAYC dataset to ;;

df.head()

,nft_id,name,asset_description,asset_contract_address,collection_name,collection_description,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,reference_id
0,2541585283815458503018281017495372409420154545...,CelebCard #84 Snoop Dogg,NaN,0x495f947276749ce646f68ac8c248420045cb7b5e,Mesmerizer,No theme. No traits. No rules. A treasure trov...,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,0x495f947276749ce646f68ac8c248420045cb7b5e-254...
1,4460258260744616717349398633449740654585409544...,3D Penguin Planet Official Collection,3D Penguin Planet Collection It was hand paint...,0x495f947276749ce646f68ac8c248420045cb7b5e,3D Penguin Planet Official Collection,3D Penguin Planet Official Collection It was h...,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,0x495f947276749ce646f68ac8c248420045cb7b5e-446...
2,1143238369460653929904223751867794581426030911...,VECTOR KAS 10 (The Relax),VECTOR KAS 10,0x495f947276749ce646f68ac8c248420045cb7b5e,Kalvintaka Collection,NaN,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,0x495f947276749ce646f68ac8c248420045cb7b5e-114...
3,5742295951199733757787373026863398866922647154...,Gooniez #1989,"A gang of 8,888 gooniez unleashed to wreak hav...",0x495f947276749ce646f68ac8c248420045cb7b5e,Metaverse GooniezNFT,"A gang of 8,888 gooniez unleashed to wreak hav...",metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,0x495f947276749ce646f68ac8c248420045cb7b5e-574...
4,1006254684714400059060824656421626037946389019...,Darumaa Soul Story.#3,Works that are created by an Extraterrestrial ...,0x495f947276749ce646f68ac8c248420045cb7b5e,Extraterrestrial Soul Darumaa,Darumaa. Not Real & Reality.\n\nWorks that are...,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...,0x495f947276749ce646f68ac8c248420045cb7b5e-100...


In [79]:
df.iloc[0]

nft_id                            2541585283815458503018281017495372409420154545...
name                                                       CelebCard #84 Snoop Dogg
asset_description                                                               NaN
asset_contract_address                   0x495f947276749ce646f68ac8c248420045cb7b5e
collection_name                                                          Mesmerizer
collection_description            No theme. No traits. No rules. A treasure trov...
collection_slug                                                          mesmerizer
collection_verification                                                       False
creator_username                                                                NaN
creator_address                          0x3830dce8b8f5543404e005776b4de22870c01d64
creator_profile_img_url           https://storage.googleapis.com/opensea-static/...
owner_username                                                          Null

## Rake Vectorizer
Vectorize words

In [80]:
def vectorize_words_in_column( column_name ):

  # initializing the new column
  new_column_name = 'Key_words_' + column_name
  df[new_column_name] = ""

  for index, row in df.iterrows():
      column_cell = row[column_name]

      
      # check if the cell has a string, to apply NLP
      if isinstance(column_cell, str):
        # instantiating Rake, by default it uses english stopwords from NLTK
        # and discards all puntuation characters as well
        r = Rake()

        # extracting the words by passing the text
        r.extract_keywords_from_text(column_cell)

        # getting the dictionary with key words as keys and their scores as values
        key_words_dict_scores = r.get_word_degrees()

        # print(key_words_dict_scores.keys())

        # assigning the key words to the new column for the corresponding NFT
        # row[new_column_name] = list(key_words_dict_scores.keys()) # this doesn't work for some reason

        df.at[index, new_column_name]=list(key_words_dict_scores.keys())

      else:
        df.at[index, new_column_name] = None

  # dropping the column_name column
  df.drop(columns = [column_name], inplace = True)


In [81]:
vectorize_words_in_column('name')
vectorize_words_in_column('asset_description')
vectorize_words_in_column('collection_name')
vectorize_words_in_column('collection_description')

df.head()

,nft_id,asset_contract_address,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,reference_id,Key_words_name,Key_words_asset_description,Key_words_collection_name,Key_words_collection_description
0,2541585283815458503018281017495372409420154545...,0x495f947276749ce646f68ac8c248420045cb7b5e,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,0x495f947276749ce646f68ac8c248420045cb7b5e-254...,"[celebcard, 84, snoop, dogg]",None,[mesmerizer],"[theme, traits, rules, treasure, trove, pure, ..."
1,4460258260744616717349398633449740654585409544...,0x495f947276749ce646f68ac8c248420045cb7b5e,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,0x495f947276749ce646f68ac8c248420045cb7b5e-446...,"[3d, penguin, planet, official, collection]","[3d, penguin, planet, collection, hand, painte...","[3d, penguin, planet, official, collection]","[3d, penguin, planet, official, collection, ha..."
2,1143238369460653929904223751867794581426030911...,0x495f947276749ce646f68ac8c248420045cb7b5e,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,0x495f947276749ce646f68ac8c248420045cb7b5e-114...,"[vector, kas, 10, relax]","[vector, kas, 10]","[kalvintaka, collection]",None
3,5742295951199733757787373026863398866922647154...,0x495f947276749ce646f68ac8c248420045cb7b5e,metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,0x495f947276749ce646f68ac8c248420045cb7b5e-574...,"[gooniez, 1989]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[metaverse, goonieznft]","[gang, 8, 888, gooniez, unleashed, wreak, havo..."
4,1006254684714400059060824656421626037946389019...,0x495f947276749ce646f68ac8c248420045cb7b5e,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...,0x495f947276749ce646f68ac8c248420045cb7b5e-100...,"[darumaa, soul, story, .#, 3]","[works, created, extraterrestrial, soul, named...","[extraterrestrial, soul, darumaa]","[darumaa, real, reality, works, created, extra..."


### Combine all key words into one column

This may come handy when comparing with trends data.

In [82]:
columns = df.columns
df['All_key_words_list'] = pd.NA
df['All_key_words_str'] = ''

for index, row in df.iterrows():
  words = []
  key_words_str=''

  for col in columns:
    if col.startswith("Key_words") and row[col] != None:
      words += row[col]
      key_words_str += ' '.join(row[col])+ ' '

  # row['All_key_words_list'] = words
  # row['All_key_words_str'] = key_words_str
  df.at[index, 'All_key_words_list'] = words
  df.at[index, 'All_key_words_str'] = key_words_str

df.head()

,nft_id,asset_contract_address,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,reference_id,Key_words_name,Key_words_asset_description,Key_words_collection_name,Key_words_collection_description,All_key_words_list,All_key_words_str
0,2541585283815458503018281017495372409420154545...,0x495f947276749ce646f68ac8c248420045cb7b5e,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,0x495f947276749ce646f68ac8c248420045cb7b5e-254...,"[celebcard, 84, snoop, dogg]",None,[mesmerizer],"[theme, traits, rules, treasure, trove, pure, ...","[celebcard, 84, snoop, dogg, mesmerizer, theme...",celebcard 84 snoop dogg mesmerizer theme trait...
1,4460258260744616717349398633449740654585409544...,0x495f947276749ce646f68ac8c248420045cb7b5e,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,0x495f947276749ce646f68ac8c248420045cb7b5e-446...,"[3d, penguin, planet, official, collection]","[3d, penguin, planet, collection, hand, painte...","[3d, penguin, planet, official, collection]","[3d, penguin, planet, official, collection, ha...","[3d, penguin, planet, official, collection, 3d...",3d penguin planet official collection 3d pengu...
2,1143238369460653929904223751867794581426030911...,0x495f947276749ce646f68ac8c248420045cb7b5e,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,0x495f947276749ce646f68ac8c248420045cb7b5e-114...,"[vector, kas, 10, relax]","[vector, kas, 10]","[kalvintaka, collection]",None,"[vector, kas, 10, relax, vector, kas, 10, kalv...",vector kas 10 relax vector kas 10 kalvintaka c...
3,5742295951199733757787373026863398866922647154...,0x495f947276749ce646f68ac8c248420045cb7b5e,metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,0x495f947276749ce646f68ac8c248420045cb7b5e-574...,"[gooniez, 1989]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[metaverse, goonieznft]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[gooniez, 1989, gang, 8, 888, gooniez, unleash...",gooniez 1989 gang 8 888 gooniez unleashed wrea...
4,1006254684714400059060824656421626037946389019...,0x495f947276749ce646f68ac8c248420045cb7b5e,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...,0x495f947276749ce646f68ac8c248420045cb7b5e-100...,"[darumaa, soul, story, .#, 3]","[works, created, ext

Create `bag_of_words`

Drop useless columns

In [83]:
df['All_key_words_str'].count()

10101

In [84]:
df.set_index('reference_id', inplace = True)   # set reference_id as the index of the dataframe
df.head()

,nft_id,asset_contract_address,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,Key_words_name,Key_words_asset_description,Key_words_collection_name,Key_words_collection_description,All_key_words_list,All_key_words_str
reference_id,,,,,,,,,,,,,,,,,,,,,,,,,
0x495f947276749ce646f68ac8c248420045cb7b5e-25415852838154585030182810174953724094201545459452816665085826154914734145537,2541585283815458503018281017495372409420154545...,0x495f947276749ce646f68ac8c248420045cb7b5e,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,"[celebcard, 84, snoop, dogg]",None,[mesmerizer],"[theme, traits, rules, treasure, trove, pure, ...","[celebcard, 84, snoop, dogg, mesmerizer, theme...",celebcard 84 snoop dogg mesmerizer theme trait...
0x495f947276749ce646f68ac8c248420045cb7b5e-44602582607446167173493986334497406545854095441469804519114872108262954631169,4460258260744616717349398633449740654585409544...,0x495f947276749ce646f68ac8c248420045cb7b5e,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,"[3d, penguin, planet, official, collection]","[3d, penguin, planet, collection, hand, painte...","[3d, penguin, planet, official, collection]","[3d, penguin, planet, official, collection, ha...","[3d, penguin, planet, official, collection, 3d...",3d penguin planet official collection 3d pengu...
0x495f947276749ce646f68ac8c248420045cb7b5e-114323836946065392990422375186779458142603091121322655546213463221980662595585,1143238369460653929904223751867794581426030911...,0x495f947276749ce646f68ac8c248420045cb7b5e,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,"[vector, kas, 10, relax]","[vector, kas, 10]","[kalvintaka, collection]",None,"[vector, kas, 10, relax, vector, kas, 10, kalv...",vector kas 10 relax vector kas 10 kalvintaka c...
0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361,5742295951199733757787373026863398866922647154...,0x495f947276749ce646f68ac8c248420045cb7b5e,metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,"[gooniez, 1989]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[metaverse, goonieznft]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[gooniez, 1989, gang, 8, 888, gooniez, unleash...",gooniez 1989 gang 8 888 gooniez unleashed wrea...
0x495f947276749ce646f68ac8c248420045cb7b5e-100625468471440005906082465642162603794638901987049655284968819362997051850753,1006254684714400059060824656421626037946389019...,0x495f947276749ce646f68ac8c248420045cb7b5e,darumaa,False,FRABATO,0xde7

In [85]:
df.drop(columns = [col for col in df.columns if not col.startswith("All_key_words")], inplace = True)
df.head()

,All_key_words_list,All_key_words_str
reference_id,,
0x495f947276749ce646f68ac8c248420045cb7b5e-25415852838154585030182810174953724094201545459452816665085826154914734145537,"[celebcard, 84, snoop, dogg, mesmerizer, theme...",celebcard 84 snoop dogg mesmerizer theme trait...
0x495f947276749ce646f68ac8c248420045cb7b5e-44602582607446167173493986334497406545854095441469804519114872108262954631169,"[3d, penguin, planet, official, collection, 3d...",3d penguin planet official collection 3d pengu...
0x495f947276749ce646f68ac8c248420045cb7b5e-114323836946065392990422375186779458142603091121322655546213463221980662595585,"[vector, kas, 10, relax, vector, kas, 10, kalv...",vector kas 10 relax vector kas 10 kalvintaka c...
0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361,"[gooniez, 1989, gang, 8, 888, gooniez, unleash...",gooniez 1989 gang 8 888 gooniez unleashed wrea...
0x495f947276749ce646f68ac8c248420045cb7b5e-100625468471440005906082465642162603794638901987049655284968819362997051850753,"[darumaa, soul, story, .#, 3, works, created, ...",darumaa soul story .# 3 works created extrater...


## Modeling

### Generic Content based matches between items

In order to detect similarities between NFT content, I need to vectorize, as I mentioned above. I decided to use CountVectorizer rather than TfIdfVectorizer for one simple reason: I need a simple frequency counter for each word in my bag_of_words column. Tf-Idf tends to give less importance to the words that are more present in the entire corpus (our whole column, in this case) which is not what we want for this application, because every word is important to detect similarity! 

*It seems like Tf-Idf would give more meaningful results* for ranking - check this later. For relevance, CountVectorizer seems to be ok.

Once I have the matrix containing the count for each word, we can apply the cosine_similarity function

In [87]:
# instantiating and generating the count matrix
count = CountVectorizer()   # used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text
count_matrix = count.fit_transform(df['All_key_words_str'])

# print(count_matrix)
count_matrix.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [88]:
count_matrix.shape

(10101, 7543)

in the count_matrix, i seems to be the NFT, j seems to be each word in the bag_of_words per NFT. the value at i, j gives the frequency of each word in the entire column of bag_of_words.

For some reason (need to find out) this frequency is needed to calculate the cosine similarity between the content in NFTs. - *The definition of similarity between two vectors u and v is, in fact, the ratio between their dot product and the product of their magnitudes.*

In [89]:
# creating a Series for the NFT reference_id so they are associated to an ordered numerical
# list I will use later to match the indexes - easy to refer the NFT reference_id by index from the array that is created (cosine_sim)
indices = pd.Series(df.index)
indices[:5]

0    0x495f947276749ce646f68ac8c248420045cb7b5e-254...
1    0x495f947276749ce646f68ac8c248420045cb7b5e-446...
2    0x495f947276749ce646f68ac8c248420045cb7b5e-114...
3    0x495f947276749ce646f68ac8c248420045cb7b5e-574...
4    0x495f947276749ce646f68ac8c248420045cb7b5e-100...
Name: reference_id, dtype: object

Check if this concern can be addressed: Having cosine_sim in-memory can be a memory constraint for larger datasets

In [90]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [91]:
# function that takes in NFT reference_id (nft_contract_address-nft_id) as input and returns the top 10 recommended NFTs
def content_based_recommendations(reference_id, cosine_sim = cosine_sim):
    
    recommended_NFTs = []
    
    # getting the index of the NFT that matches the reference_id
    idx = indices[indices == reference_id].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar NFTs
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the reference_ids of the best 10 matching NFT
    for i in top_10_indexes:
        recommended_NFTs.append(list(df.index)[i])
        
    return recommended_NFTs

### Trends based Content maching

In [ ]:
trending_phrases = ['ROMCOM SERIES FOR HAESOO', 'Golden Corral', 'Wordle 228', 'Tietê', 'BOP TRACKLIST', '伊東純也', 'JIKJIN FOCUS FILM', 'NINJAPAN 2021', '一気読み', 'ムササビ', 'オーストラリア', 'WONPIL Concept Photo 3', 'Benzine 39', 'あーりん', 'البليهي', 'Kraft', '花枝ちゃん', 'HAPPY BIRTHDAY GEONU', 'ちゃん誕生日', 'サウジアラビア', '도영듀스', '凛ちゃん', 'SECRETCRUSH TRAILER', 'Best Entertainer Jungkook', 'UM CORPO DE VERÃO', 'Quito', 'Damn Brady', 'Schefter', '日村さん', 'SMILE FOR MEW', 'HBCUs', 'もやとらっしー', 'Lucas', '拷問部屋', 'Wordle 244 X', '視聴覚室', '子供部屋', 'Gustavo', '警視庁初摘発', '日本人の覚醒剤製造', '暴力団幹部とYouTuber組員ら逮捕', 'Donda 2', 'Tiago', 'ベランダ', 'Peaky Peaky', '당신 이야기', '대신 전해', '¥930,000', 'RED VELVET MARCH COMEBACK', 'Embiid', 'Jack Harlow', 'sabrina', 'Nonku', 'Giannis', '国民負担率', '¥115,000', '鬱アニメ', "GUESS WHO'S BACK", 'TBS NEWS', '¥139,000', 'sungas', '超大作怪獣映画', 'Middleton', '男性医師の有罪判決', 'Barão da Piscadinha', '自爆デッキ', 'Stem Player', 'はしもっちゃん', 'Aグループ', 'Eslo', 'あやかし譚', 'おててまもる君', 'ミラキュラス', 'Yeat']
# phrases contains all trending words

In [62]:
import pprint   # well-formatted print output

pp = pprint.PrettyPrinter(indent=1)

In [92]:
pre_processed_twitter_trends = []

bag_of_trends_phrases = []
min_tweet_volume = None

for trend in twitter_trends:
  if trend['name'] not in bag_of_trends_phrases :
    # ignore duplicates (twitter API bug? sometimes trends are duplicated) 
    bag_of_trends_phrases.append(trend['name'])

    pre_processed_trend = trend

    #  remove hashtags
    if trend['name'][0:1] == '#':
      # remove hashtag
      pre_processed_trend['name'] = trend['name'][1:]


      #  split words that are combined - usually happens for hashtags

    # update min_tweet_volume
    if trend['tweet_volume']:
      if min_tweet_volume == None:
        # first trend which has a tweet_volume
        min_tweet_volume = trend['tweet_volume']
      elif trend['tweet_volume'] < min_tweet_volume:
        # update min_tweet_volume
        min_tweet_volume = trend['tweet_volume']

    # TODO: apply sentiment analysis on url page - extract sentiment



    #  convert name to lower case
    pre_processed_trend['name'] = pre_processed_trend['name'].lower()

    pre_processed_twitter_trends.append(pre_processed_trend)

In [94]:
# add min tweet volume for tweets with no volume
for index, trend in enumerate(pre_processed_twitter_trends):
  if trend['tweet_volume'] == None:
      pre_processed_twitter_trends[index]['tweet_volume'] = min_tweet_volume - 1


# print(pre_processed_twitter_trends)
# pp.pprint(pre_processed_twitter_trends)

Calculate Median Tweet volume

An impact score can be calculated separately as well.

In [95]:
import statistics

# add all tweets with tweet volumes into an array
tweet_volumes_array = []
for tweet in pre_processed_twitter_trends:
  if tweet['tweet_volume'] != None:
    tweet_volumes_array.append(tweet['tweet_volume'])

print("tweet_volumes_array:", tweet_volumes_array)

# calculate median tweet volume
median_tweet_volume = statistics.median(tweet_volumes_array)
print(median_tweet_volume)

tweet_volumes_array: [322831, 108136, 10273, 10273, 20144, 61377, 21343, 10273, 10273, 10273, 21931, 10273, 25839, 10273, 10273, 35034, 19631, 12599, 10273, 21827, 11473, 10273, 10273, 17690, 10366, 165262, 10273, 10273, 10273, 11387, 10273, 11340, 25023, 40599, 10273, 18118, 10273, 10273, 17573, 17557, 10273, 10273, 10273, 47811, 126678, 10273, 55494, 10273, 73660, 10274]
10853.0


Give a score for each row (NFT) based on the matching content

In [ ]:
df['trend_score'] = pd.NA
not_interested_trends = []  # defined by admin/ each user?

for index, row in df.iterrows():
  new_trend_score = 0

  for trend in pre_processed_twitter_trends:

    if trend['name'] in row['All_key_words_list']:
    # if the content matches
      volume = trend['tweet_volume']
      new_trend_score += (volume/ median_tweet_volume)

      print(trend, "\n",row)

  # old_trend_score = row['trend_score']
  # new_trend_score = old_trend_score + 

  df.at[index, 'trend_score'] = new_trend_score

df.head()

In [97]:
def trends_based_recommendations():
  top_trending_df = df.sort_values(by=['trend_score'], ascending=False)

  return top_trending_df


## Testing

In [ ]:
content_based_recommendations('0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361')

['0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444682273409728513',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444687770967867393',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444669079270195201',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444641591479500801',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444656984642289665',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444666880246939649',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444699865595772929',


In [98]:
# trends_based_recommendations('0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361')

top_trending_nfts_df = trends_based_recommendations()

top_trending_nfts_df.head()

,All_key_words_list,All_key_words_str,trend_score
reference_id,,,
0x1c2cd50f9efb463bdd2ec9e36772c14a8d1658b3-15332,"[bees, deluxe, bees, deluxe, collection, 20, 7...",bees deluxe bees deluxe collection 20 700 nice...,1.966553
0x1c2cd50f9efb463bdd2ec9e36772c14a8d1658b3-14369,"[bees, deluxe, bees, deluxe, collection, 20, 7...",bees deluxe bees deluxe collection 20 700 nice...,1.966553
0x1c2cd50f9efb463bdd2ec9e36772c14a8d1658b3-15332,"[bees, deluxe, bees, deluxe, collection, 20, 7...",bees deluxe bees deluxe collection 20 700 nice...,1.966553
0x1c2cd50f9efb463bdd2ec9e36772c14a8d1658b3-15332,"[bees, deluxe, bees, deluxe, collection, 20, 7...",bees deluxe bees deluxe collection 20 700 nice...,1.966553
0x1c2cd50f9efb463bdd2ec9e36772c14a8d1658b3-14369,"[bees, deluxe, bees, deluxe, collection, 20, 7...",bees deluxe bees deluxe collection 20 700 nice...,1.966553


# Custom Integration to rerank using Social Trends

In [ ]:
trending_phrases = ['ROMCOM SERIES FOR HAESOO', 'Golden Corral', 'Wordle 228', 'Tietê', 'BOP TRACKLIST', '伊東純也', 'JIKJIN FOCUS FILM', 'NINJAPAN 2021', '一気読み', 'ムササビ', 'オーストラリア', 'WONPIL Concept Photo 3', 'Benzine 39', 'あーりん', 'البليهي', 'Kraft', '花枝ちゃん', 'HAPPY BIRTHDAY GEONU', 'ちゃん誕生日', 'サウジアラビア', '도영듀스', '凛ちゃん', 'SECRETCRUSH TRAILER', 'Best Entertainer Jungkook', 'UM CORPO DE VERÃO', 'Quito', 'Damn Brady', 'Schefter', '日村さん', 'SMILE FOR MEW', 'HBCUs', 'もやとらっしー', 'Lucas', '拷問部屋', 'Wordle 244 X', '視聴覚室', '子供部屋', 'Gustavo', '警視庁初摘発', '日本人の覚醒剤製造', '暴力団幹部とYouTuber組員ら逮捕', 'Donda 2', 'Tiago', 'ベランダ', 'Peaky Peaky', '당신 이야기', '대신 전해', '¥930,000', 'RED VELVET MARCH COMEBACK', 'Embiid', 'Jack Harlow', 'sabrina', 'Nonku', 'Giannis', '国民負担率', '¥115,000', '鬱アニメ', "GUESS WHO'S BACK", 'TBS NEWS', '¥139,000', 'sungas', '超大作怪獣映画', 'Middleton', '男性医師の有罪判決', 'Barão da Piscadinha', '自爆デッキ', 'Stem Player', 'はしもっちゃん', 'Aグループ', 'Eslo', 'あやかし譚', 'おててまもる君', 'ミラキュラス', 'Yeat']
# phrases contains all trending words

Phrase-Based Matching

This may be useful for a question asking platform, not for NFT recommendations

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

In [ ]:
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)

# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in trending_phrases]
matcher.add("TerminologyList", patterns)


Content-based matching